In [1]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf
import model_fsolve_v2 as modf2

import scipy.optimize as sco
import scipy.interpolate as sci

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import os
import plotly.graph_objects as go


In [3]:
Vdot_list = [0.01, 0.05, 0.1, 0.5] 

SR_fp = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\CFD\V4.5_1channel_SR'

In [13]:
p_op = 1e5
T_op = 25+273.15

In [15]:
DP_list = []
DP_theo_list = []

for Vdot in Vdot_list:

    folder_path_case = os.path.join(SR_fp, 'parametric_laminar_try0', f'case_{str(Vdot)}Lh')

    # report_names = ['report_outlet_sp', 'report_inlet_sp', 'report_outlet_mdot']
    # for report_name in report_names:
    #     jg.change_report_file_path(tui, report_name, os.path.join(folder_path_case, f"{report_name}.out"))

    Vdot_m3s = Vdot * 1e-3 / 3600 # m3/s
    Ac = 0.0000055427 # m2
    V = Vdot_m3s / Ac # m/s

    sp_df = pd.read_csv(os.path.join(folder_path_case, 'sp_report.csv'), sep=',')

    DP = sp_df.loc[sp_df['Component'] == 'inlet', 'sp'].values[0] - sp_df.loc[sp_df['Component'] == 'outlet', 'sp'].values[0]

    DP_list.append(DP)

    # jg.change_named_expression(tui, 'V_inlet', V, 'm/s')

    channel = hxhy.duct('rectangular', h = 0.002, w = 0.0028, L = 1.603) # kPa
    PL_guess = 1000 * channel.regular_PL(Vdot, fluid_dict= {'name':'MPG', 'glycol_rate' : 0.4}, p=p_op, T=T_op) # Pa

    DP_theo_list.append(PL_guess)

    # jg.standard_initialization(tui, 'inlet', PL_guess, coeff_x*V, coeff_y*V, coeff_z*V)

    # solver.solution.run_calculation.iterate(number_of_iterations=nb_it)

    # jg.write_report(tui, 'mdot', folder_path_case, 'mdot_report')
    # jg.write_report(tui, 'sp', folder_path_case, 'sp_report')
    # jg.write_residuals_file(tui, folder_path_case, 'residuals')
    # jg.write_data(tui, folder_path_case, f'case_{str(Vdot)}Lh')

In [ ]:
plt.plot(Vdot_list, DP_list, label='CFD')
plt.plot(Vdot_list, DP_theo_list, label='Theoretical')
plt.legend()